In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Carga de datos
archivo = "../DEPURATION/datos_limpios.xlsx"  # Cambia al nombre de tu archivo
df = pd.read_excel(archivo)

# Preprocesamiento

# Crear variable binaria para accidente: 1 si TOTAL_VICTIMAS > 0, de lo contrario 0
df['accidente_ocurrido'] = (df['TOTAL_VICTIMAS'] > 0).astype(int)

# Seleccionar variables de interés
features = ['MES', 'DIA_MES', 'METEO_ORIGEN', 'METEO_DESTINO', 'provincia_origen', 'provincia_destino']
X = df[features]
y = df['accidente_ocurrido']


# Codificar variables categóricas
X_encoded = pd.get_dummies(X, columns=['METEO_ORIGEN', 'METEO_DESTINO', 'provincia_origen', 'provincia_destino'], drop_first=True)

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Entrenar el modelo de Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probabilidad de accidente

# Evaluación del modelo
print("Exactitud:", accuracy_score(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_pred_proba))
print("\nInforme de clasificación:\n", classification_report(y_test, y_pred))


Exactitud: 0.9734291094218565
AUC-ROC: 0.9965846036793008

Informe de clasificación:
               precision    recall  f1-score   support

           0       0.98      0.98      0.98     98563
           1       0.94      0.95      0.94     30488

    accuracy                           0.97    129051
   macro avg       0.96      0.96      0.96    129051
weighted avg       0.97      0.97      0.97    129051



In [11]:
X_test_original = X.loc[X_test.index]  # Seleccionamos las filas originales correspondientes a X_test
resultado = X_test_original.copy()
resultado['PROBABILIDAD_DE_ACCIDENTE'] = y_pred_proba
resultado.to_excel('predicciones_accidentes.xlsx', index=False)
resultado.head()


,MES,DIA_MES,METEO_ORIGEN,METEO_DESTINO,provincia_origen,provincia_destino,PROBABILIDAD_DE_ACCIDENTE
517781,10,19,NaN,NaN,18,34,0.05
306787,6,24,NaN,NaN,47,49,0.02
540391,11,1,NaN,NaN,13,36,0.04
129821,3,18,1.0,NaN,1,40,0.66
298285,6,20,NaN,NaN,16,37,0.00
